# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('ETL_data', engine)
X = df.iloc[:,1]
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
     
pipeline = Pipeline(
    [
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('mclf', MultiOutputClassifier(RandomForestClassifier()))
    ]
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

In [9]:
category_names = Y.columns.astype('str')

ind = 0
for cat in category_names:
    print(f'{cat} category summary:')
    print(classification_report(y_test[cat].values, y_pred[:, ind]))
    print('------------------------------------------------------')
    
    ind += 1

related category summary:
             precision    recall  f1-score   support

          0       0.61      0.34      0.44      1545
          1       0.82      0.93      0.87      4965
          2       1.00      0.09      0.16        35

avg / total       0.77      0.79      0.77      6545

------------------------------------------------------
request category summary:
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      5424
          1       0.84      0.38      0.53      1121

avg / total       0.88      0.88      0.86      6545

------------------------------------------------------
offer category summary:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6514
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6545

------------------------------------------------------
aid_related category summary:
             precision    recal

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('mclf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_

In [17]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'mclf__estimator__warm_start':[True, False],
        'mclf__estimator__max_features':['auto','log2'],
        'mclf__estimator__criterion':['gini', 'entropy'],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=2, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print(f'y_pred type is {y_pred.dtype}')
print(f'y_pred type is {y_test.dtypes}')

Fitting 2 folds for each of 864 candidates, totalling 1728 fits
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, mclf__estimator__criterion=gini, mclf__estimator__max_features=auto, mclf__estimator__warm_start=True 


TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

In [11]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

category_names = Y.columns.astype('str')

ind = 0
for cat in category_names:
    print(f'{cat} category summary:')
    print(classification_report(y_test[cat].values, y_pred[:, ind]))
    print('------------------------------------------------------')
    
    ind += 1

Fitting 2 folds for each of 864 candidates, totalling 1728 fits
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, mclf__estimator__criterion=gini, mclf__estimator__max_features=auto, mclf__estimator__warm_start=True 


TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.